In [1]:
!wget "https://he-public-data.s3.ap-southeast-1.amazonaws.com/shell_dataset.zip"

--2021-11-07 07:02:15--  https://he-public-data.s3.ap-southeast-1.amazonaws.com/shell_dataset.zip
Resolving he-public-data.s3.ap-southeast-1.amazonaws.com (he-public-data.s3.ap-southeast-1.amazonaws.com)... 52.219.125.43
Connecting to he-public-data.s3.ap-southeast-1.amazonaws.com (he-public-data.s3.ap-southeast-1.amazonaws.com)|52.219.125.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3467218047 (3.2G) [binary/octet-stream]
Saving to: ‘shell_dataset.zip’

shell_dataset.zip   100%[===================>]   3.23G  13.1MB/s    in 4m 13s  

2021-11-07 07:06:29 (13.1 MB/s) - ‘shell_dataset.zip’ saved [3467218047/3467218047]



In [2]:
!unzip -q "shell_dataset.zip"
!unzip -q "dataset/train.zip"
!unzip -q "dataset/test.zip"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.api import VAR

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
tdf = pd.read_csv('dataset/test.csv')

In [5]:
tdf = tdf.set_index('scenario_set')

In [6]:
path = 'test/'+str(232)+'/weather_data.csv'
df = pd.read_csv(path)
df.columns

Index(['Time [Mins]', 'Global CMP22 (vent/cor) [W/m^2]', 'Direct sNIP [W/m^2]',
       'Azimuth Angle [degrees]', 'Tower Dry Bulb Temp [deg C]',
       'Tower Wet Bulb Temp [deg C]', 'Tower Dew Point Temp [deg C]',
       'Tower RH [%]', 'Total Cloud Cover [%]', 'Peak Wind Speed @ 6ft [m/s]',
       'Avg Wind Direction @ 6ft [deg from N]', 'Station Pressure [mBar]',
       'Precipitation (Accumulated) [mm]', 'Snow Depth [cm]', 'Moisture',
       'Albedo (CMP11)'],
      dtype='object')

In [7]:
df.corr()['Total Cloud Cover [%]']

Time [Mins]                             -0.341169
Global CMP22 (vent/cor) [W/m^2]          0.167205
Direct sNIP [W/m^2]                     -0.182232
Azimuth Angle [degrees]                 -0.384575
Tower Dry Bulb Temp [deg C]              0.271967
Tower Wet Bulb Temp [deg C]             -0.155039
Tower Dew Point Temp [deg C]            -0.320068
Tower RH [%]                            -0.302800
Total Cloud Cover [%]                    1.000000
Peak Wind Speed @ 6ft [m/s]              0.056552
Avg Wind Direction @ 6ft [deg from N]   -0.067952
Station Pressure [mBar]                 -0.209176
Precipitation (Accumulated) [mm]        -0.290730
Snow Depth [cm]                         -0.100890
Moisture                                -0.308059
Albedo (CMP11)                           0.364291
Name: Total Cloud Cover [%], dtype: float64

In [11]:
for i in range(1,301):
  path = 'test/'+str(i)+'/weather_data.csv'
  df = pd.read_csv(path)
  todays_date = datetime.now().date()
  index = pd.date_range('2020-01-01', periods=361, freq='1min')
  df = df.set_index(index[0:361])
  df.drop('Time [Mins]',inplace=True,axis=1)
  df['Total Cloud Cover [%]'] = [np.NaN if (per < 0 or per > 100) else per for per in df['Total Cloud Cover [%]']]
  df['Total Cloud Cover [%]'] = df['Total Cloud Cover [%]'].interpolate(method='polynomial', order=2)
  df['Total Cloud Cover [%]'].fillna(df['Total Cloud Cover [%]'].max(),inplace = True)
  df = df[['Direct sNIP [W/m^2]', 'Total Cloud Cover [%]']]
  model = VAR(df)
  history = model.fit(2)
  lag_order = history.k_ar
  forecast_input = df.values[-lag_order:]
  index = pd.date_range('2020-01-01 06:00:00', periods=121, freq='1min')
  n_forecast = history.forecast(forecast_input,steps=121)  
  forecast = pd.DataFrame(n_forecast, index=index, columns=df.columns)
  forecast['Total Cloud Cover [%]'] = [0 if (per < 0) else per for per in forecast['Total Cloud Cover [%]']]
  forecast['Total Cloud Cover [%]'] = [100 if (per > 100) else per for per in forecast['Total Cloud Cover [%]']]
  tdf.loc[i,'30_min_horizon'] = forecast.loc['2020-01-01 06:30:00','Total Cloud Cover [%]']
  tdf.loc[i,'60_min_horizon'] = forecast.loc['2020-01-01 07:00:00','Total Cloud Cover [%]']
  tdf.loc[i,'90_min_horizon'] = forecast.loc['2020-01-01 07:30:00','Total Cloud Cover [%]']
  tdf.loc[i,'120_min_horizon'] = forecast.loc['2020-01-01 08:00:00','Total Cloud Cover [%]']

In [12]:
tdf

,30_min_horizon,60_min_horizon,90_min_horizon,120_min_horizon
scenario_set,,,,
1,98.049743,97.369015,96.815061,96.359585
2,30.332041,26.865647,25.349410,24.686350
3,98.858350,98.933549,98.938937,98.939323
4,50.568988,48.844118,47.586166,46.668747
5,19.127955,18.842190,18.744232,18.710653
...,...,...,...,...
296,12.610083,12.666616,12.680218,12.686184
297,57.921445,59.691877,61.206035,62.501014
298,14.144464,14.510840,14.740920,14.913532


In [13]:
tdf.to_csv('sol.csv')